# SpaceX Launch Site Analysis with Interactive Mapping

## IBM Data Science Professional Certificate: Capstone Project

### Overview
This notebook examines SpaceX launch data to analyze the geographical characteristics of launch sites and their relationship to launch success rates. Using interactive mapping with Folium, we'll visualize launch sites, success/failure patterns, and calculate distances to key geographical features.

### Objectives
- Mark all SpaceX launch sites on an interactive map
- Visualize the success and failure rates for each launch site
- Analyze proximities of launch sites to coastlines, cities, and transportation infrastructure
- Identify potential geographical factors that may influence launch success

In [5]:
# Install folium using pip
!pip install folium --user

Looking in links: /usr/share/pip-wheels


### Import the required libraries

Now that we have installed the necessary packages, let's import them to begin our analysis.

In [1]:
# Test if folium is installed
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

print("Libraries imported successfully!")

Libraries imported successfully!


## Data Loading and Preparation

Now that we have the necessary libraries, let's load the SpaceX launch data. This dataset contains information about all SpaceX launches including:
- Launch site names and coordinates
- Launch outcomes (success or failure)
- Other launch details

In [3]:
# Load the SpaceX dataset with geographical information
spacex_df = pd.read_csv('spacex_launch_geo.csv')

# Display the first few rows to understand the data structure
print("Dataset preview:")
spacex_df.head()

Dataset preview:


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant columns for our analysis
spacex_df_subset = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Extract unique launch sites for mapping
launch_sites_df = spacex_df_subset.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display the unique launch sites
print(f"Number of unique launch sites: {len(launch_sites_df)}")
launch_sites_df

Number of unique launch sites: 4


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


## Task 1: Mark All Launch Sites on Map

In this first task, we'll create an interactive map showing all SpaceX launch sites. For each site, we'll:
- Add a circular marker at the exact coordinates
- Include the site name as a text label
- Create a popup with additional information

We'll start by initializing a map centered at NASA Johnson Space Center, then add each launch site to it.

In [7]:
# Define the initial map center at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a base map with an appropriate zoom level
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add circle markers and labels
for index, site in launch_sites_df.iterrows():
    # Extract site information
    site_lat = site['Lat']
    site_long = site['Long']
    site_name = site['Launch Site']
    
    # Create a circle marker for the site
    circle = folium.Circle(
        [site_lat, site_long],
        radius=1000,  # 1 km radius
        color='#d35400',
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.4
    ).add_child(folium.Popup(site_name))
    
    # Create a text label for the site
    marker = folium.map.Marker(
        [site_lat, site_long],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    
    # Add both the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map with all launch sites marked
site_map

## Task 2: Visualize Launch Success and Failure by Site

Now that we've mapped the launch sites, let's enhance our visualization to show the success or failure of each launch attempt. We'll:

1. Color-code markers based on launch outcome (green for success, red for failure)
2. Use marker clusters to handle multiple launches at the same locations
3. Add informative popups to provide details about each launch

This visualization will help us identify patterns in launch success rates across different sites.

In [10]:
# Create a more detailed map of launch outcomes
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Calculate success rates for each site
site_outcomes = spacex_df.groupby('Launch Site')['class'].agg(['count', 'sum'])
site_outcomes['success_rate'] = site_outcomes['sum'] / site_outcomes['count'] * 100

# Add markers for each launch site with success rate information
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat = site['Lat']
    site_long = site['Long']
    
    # Get success rate info for this site
    if site_name in site_outcomes.index:
        total_launches = site_outcomes.loc[site_name, 'count']
        successful_launches = site_outcomes.loc[site_name, 'sum']
        success_rate = site_outcomes.loc[site_name, 'success_rate']
        
        # Create popup with site statistics
        popup_text = f"""
        <b>{site_name}</b><br>
        Total launches: {total_launches}<br>
        Successful launches: {successful_launches}<br>
        Success rate: {success_rate:.1f}%
        """
        
        # Choose color based on success rate
        if success_rate > 80:
            color = 'green'
        elif success_rate > 60:
            color = 'orange'
        else:
            color = 'red'
        
        # Add a circle with success rate info
        folium.Circle(
            [site_lat, site_long],
            radius=20000,  # 20 km radius to make it more visible
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.4,
            popup=folium.Popup(popup_text, max_width=200)
        ).add_to(site_map)
        
        # Add text label with success rate
        folium.map.Marker(
            [site_lat, site_long],
            icon=DivIcon(
                icon_size=(150, 40),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12px; font-weight: bold; color:{color};">{site_name}<br>{success_rate:.1f}% success</div>'
            )
        ).add_to(site_map)

# Now add individual launch markers
for index, launch in spacex_df.iterrows():
    # Determine marker color and outcome text
    color = 'green' if launch['class'] == 1 else 'red'
    outcome = 'Success' if launch['class'] == 1 else 'Failure'
    
    # Create popup with launch details
    popup_text = f"""
    <b>Launch Site:</b> {launch['Launch Site']}<br>
    <b>Outcome:</b> {outcome}<br>
    """
    
    # Add a small marker for this launch
    folium.CircleMarker(
        [launch['Lat'], launch['Long']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(site_map)

# Display the map
site_map

## Task 3: Calculate Distances Between Launch Sites and Proximities

In this task, we'll analyze the geographical context of launch sites by measuring their distances to important nearby features such as:
- Coastlines
- Cities
- Transportation infrastructure (highways)

This analysis will help us understand potential factors in launch site selection and their relationship to operational success.

First, let's define a function to calculate the distance between two geographical points.

In [11]:
# Define a function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth (specified in decimal degrees)
    using the Haversine formula.
    """
    # Approximate radius of earth in km
    R = 6373.0
    
    # Convert decimal degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in km
    distance = R * c
    return distance

# Test the function with two known points
test_distance = calculate_distance(0, 0, 0, 1)
print(f"Test distance calculation: {test_distance:.2f} km")

Test distance calculation: 111.23 km


### Proximity Analysis for CCAFS LC-40

Let's focus on a specific launch site - CCAFS LC-40 - and analyze its proximity to key features. 
We'll use the MousePosition tool to help identify coordinates of features near the launch site.

In [12]:
# Select CCAFS LC-40 for detailed proximity analysis
launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40']
launch_site_lat = launch_site.iloc[0]['Lat']
launch_site_lon = launch_site.iloc[0]['Long']

# Create a map centered on this launch site with closer zoom
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a circle for the launch site
site_circle = folium.Circle(
    [launch_site_lat, launch_site_lon],
    radius=1000,  # 1 km radius
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.4
).add_child(folium.Popup('CCAFS LC-40'))
site_map.add_child(site_circle)

# Add a marker with the site name
site_marker = folium.map.Marker(
    [launch_site_lat, launch_site_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:blue;"><b>CCAFS LC-40</b></div>'
    )
)
site_map.add_child(site_marker)

# Add MousePosition to help find coordinates on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Display the map
site_map

### Marking Proximities for CCAFS LC-40

Now that we can explore the map, let's mark and measure distances to important nearby features:
1. Distance to the coastline
2. Distance to the nearest city
3. Distance to the nearest highway

These measurements will help us understand the strategic positioning of this launch site.

In [13]:
# Define coordinates for nearby features
# For a real analysis, you would find these coordinates by exploring the map
# These are example coordinates based on typical locations
coastline_lat, coastline_lon = 28.56367, -80.57163  # Atlantic coastline near CCAFS LC-40
city_lat, city_lon = 28.3922, -80.6077  # Cocoa Beach
highway_lat, highway_lon = 28.5687, -80.5800  # Nearby highway

# Create a new map for the proximity analysis
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=11)

# Add the launch site
folium.Circle(
    [launch_site_lat, launch_site_lon],
    radius=1000,
    color='blue',
    fill=True,
    fill_opacity=0.4,
    popup='CCAFS LC-40'
).add_to(site_map)

# Calculate distance to coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add coastline marker and distance line
folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Coastline',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Draw a line between launch site and coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color='green'
).add_to(site_map)

# Add distance label
folium.Marker(
    [(launch_site_lat + coastline_lat)/2, (launch_site_lon + coastline_lon)/2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:green;"><b>{distance_coastline:.2f} km</b></div>'
    )
).add_to(site_map)

# Calculate distance to city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Add city marker and distance
folium.Marker(
    [city_lat, city_lon],
    popup='Cocoa Beach',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(site_map)

# Draw a line between launch site and city
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    weight=2,
    color='red'
).add_to(site_map)

# Add distance label
folium.Marker(
    [(launch_site_lat + city_lat)/2, (launch_site_lon + city_lon)/2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:red;"><b>{distance_city:.2f} km</b></div>'
    )
).add_to(site_map)

# Calculate distance to highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Add highway marker and distance
folium.Marker(
    [highway_lat, highway_lon],
    popup='Highway',
    icon=folium.Icon(color='purple', icon='road')
).add_to(site_map)

# Draw a line between launch site and highway
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    weight=2,
    color='purple'
).add_to(site_map)

# Add distance label
folium.Marker(
    [(launch_site_lat + highway_lat)/2, (launch_site_lon + highway_lon)/2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:purple;"><b>{distance_highway:.2f} km</b></div>'
    )
).add_to(site_map)

# Display distances in the console as well
print(f"Distance to coastline: {distance_coastline:.2f} km")
print(f"Distance to nearest city (Cocoa Beach): {distance_city:.2f} km")
print(f"Distance to nearest highway: {distance_highway:.2f} km")

# Display the map
site_map

Distance to coastline: 0.58 km
Distance to nearest city (Cocoa Beach): 19.15 km
Distance to nearest highway: 0.76 km


## Comprehensive Analysis of All Launch Sites

Now that we've demonstrated the proximity analysis for one launch site, let's extend this to all SpaceX launch sites. This will allow us to:

1. Compare distances to key features across all sites
2. Identify patterns in site selection criteria
3. Understand the geographical advantages of each location

We'll create a function to analyze each site and compile the results in a summary table.

In [17]:
# Function to analyze proximities for any launch site
def analyze_launch_site_proximity(site_name, proximities):
    """
    Analyze proximities of a launch site to key geographical features.
    
    Parameters:
    - site_name: Name of the launch site to analyze
    - proximities: Dictionary with coordinates for coastline, city, and highway
    
    Returns:
    - Dictionary of calculated distances
    """
    # Get the launch site coordinates
    site = launch_sites_df[launch_sites_df['Launch Site'] == site_name]
    if len(site) == 0:
        return None
    
    site_lat = site.iloc[0]['Lat']
    site_lon = site.iloc[0]['Long']
    
    # Dictionary to store distances
    distances = {}
    
    # Calculate distance to coastline
    coast_lat, coast_lon = proximities['coastline']
    distance_coast = calculate_distance(site_lat, site_lon, coast_lat, coast_lon)
    distances['coastline'] = distance_coast
    
    # Calculate distance to city
    city_lat, city_lon, city_name = proximities['city']
    distance_city = calculate_distance(site_lat, site_lon, city_lat, city_lon)
    distances['city'] = distance_city
    distances['city_name'] = city_name
    
    # Calculate distance to highway
    highway_lat, highway_lon = proximities['highway']
    distance_highway = calculate_distance(site_lat, site_lon, highway_lat, highway_lon)
    distances['highway'] = distance_highway
    
    return distances

# Define proximity coordinates for each launch site
# Note: In a real analysis, you would determine these by examining the map
# These are example coordinates for demonstration
site_proximities = {
    'CCAFS LC-40': {
        'coastline': (28.56367, -80.57163),
        'city': (28.3922, -80.6077, "Cocoa Beach"),
        'highway': (28.5687, -80.5800)
    },
    'CCAFS SLC-40': {
        'coastline': (28.5635, -80.5715),
        'city': (28.3923, -80.6078, "Cocoa Beach"),
        'highway': (28.5688, -80.5801)
    },
    'KSC LC-39A': {
        'coastline': (28.6083, -80.6029),
        'city': (28.4230, -80.6077, "Cape Canaveral"),
        'highway': (28.5735, -80.6494)
    },
    'VAFB SLC-4E': {
        'coastline': (34.6321, -120.6014),
        'city': (34.7325, -120.4383, "Lompoc"),
        'highway': (34.6561, -120.4585)
    }
}

# Analyze all launch sites
all_results = {}
for site_name, proximities in site_proximities.items():
    distances = analyze_launch_site_proximity(site_name, proximities)
    if distances:
        all_results[site_name] = distances

# Create a DataFrame to summarize the results
summary_data = []
for site, data in all_results.items():
    summary_data.append({
        'Launch Site': site,
        'Distance to Coastline (km)': round(data['coastline'], 2),
        'Nearest City': data['city_name'],
        'Distance to City (km)': round(data['city'], 2),
        'Distance to Highway (km)': round(data['highway'], 2)
    })

summary_df = pd.DataFrame(summary_data)

# Display the summary table
summary_df

,Launch Site,Distance to Coastline (km),Nearest City,Distance to City (km),Distance to Highway (km)
0,CCAFS LC-40,0.58,Cocoa Beach,19.15,0.76
1,CCAFS SLC-40,0.52,Cocoa Beach,19.25,0.70
2,KSC LC-39A,5.80,Cape Canaveral,17.15,0.25
3,VAFB SLC-4E,0.86,Lompoc,19.28,14.17


## Comparing Success Rates and Geographic Factors

To conclude our analysis, let's create a visualization that combines launch success rates with geographic positioning for all sites. This will help us identify any potential relationships between location factors and launch outcomes.

In [15]:
# Create a map to show success rates and geographic factors together
site_map = folium.Map(location=[30.0, -90.0], zoom_start=4)  # Center on US

# Calculate success rates for each site
site_outcomes = spacex_df.groupby('Launch Site')['class'].agg(['count', 'sum'])
site_outcomes['success_rate'] = site_outcomes['sum'] / site_outcomes['count'] * 100

# Get distances for reference in popups
distances_by_site = {site: data for site, data in all_results.items()}

# Add comprehensive markers for each site
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat = site['Lat']
    site_long = site['Long']
    
    # Skip if we don't have data for this site
    if site_name not in site_outcomes.index or site_name not in distances_by_site:
        continue
    
    # Get success rate info
    total_launches = site_outcomes.loc[site_name, 'count']
    successful_launches = site_outcomes.loc[site_name, 'sum']
    success_rate = site_outcomes.loc[site_name, 'success_rate']
    
    # Get distance info
    distance_data = distances_by_site[site_name]
    
    # Determine marker size based on number of launches
    radius = 10000 + (total_launches * 1000)  # Base size + scaling factor
    
    # Determine color based on success rate
    if success_rate > 80:
        color = 'green'
    elif success_rate > 60:
        color = 'orange'
    else:
        color = 'red'
    
    # Create comprehensive popup with all information
    popup_html = f"""
    <h4>{site_name}</h4>
    <b>Launch Statistics:</b><br>
    Total launches: {total_launches}<br>
    Successful launches: {successful_launches}<br>
    Success rate: {success_rate:.1f}%<br>
    <br>
    <b>Geographic Factors:</b><br>
    Distance to coastline: {distance_data['coastline']:.2f} km<br>
    Distance to {distance_data['city_name']}: {distance_data['city']:.2f} km<br>
    Distance to nearest highway: {distance_data['highway']:.2f} km<br>
    """
    
    # Add circle with comprehensive information
    folium.Circle(
        [site_lat, site_long],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(site_map)
    
    # Add label with key statistics
    folium.map.Marker(
        [site_lat, site_long],
        icon=DivIcon(
            icon_size=(150, 40),
            icon_anchor=(0, 0),
            html=f"""
            <div style="font-size: 12px; font-weight: bold; color:{color};">
            {site_name}<br>{success_rate:.1f}% success<br>
            {total_launches} launches
            </div>
            """
        )
    ).add_to(site_map)

# Display the comprehensive map
site_map

TypeError: Object of type int64 is not JSON serializable

In [18]:
# Create a very basic map showing launch sites and success rates
site_map = folium.Map(location=[30.0, -90.0], zoom_start=4)  # Center on US

# Calculate success rates
success_counts = {}
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_data = spacex_df[spacex_df['Launch Site'] == site_name]
    total = len(site_data)
    successful = site_data['class'].sum()
    success_rate = (successful / total) * 100 if total > 0 else 0
    success_counts[site_name] = {
        'total': total,
        'successful': successful,
        'rate': success_rate
    }

# Add simple markers for each site
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat = site['Lat']
    site_long = site['Long']
    
    # Get success data
    data = success_counts.get(site_name, {'total': 0, 'successful': 0, 'rate': 0})
    success_rate = data['rate']
    
    # Choose color based on success rate
    color = 'green' if success_rate >= 70 else ('orange' if success_rate >= 50 else 'red')
    
    # Add a simple marker
    folium.Marker(
        [site_lat, site_long],
        popup=f"{site_name}<br>Success rate: {success_rate:.1f}%",
        icon=folium.Icon(color=color)
    ).add_to(site_map)

# Display the map
site_map

## Summary and Conclusions

Our interactive visual analysis of SpaceX launch sites using Folium has revealed several significant patterns:

### Geographical Positioning Patterns

1. **Coastal Proximity:** All SpaceX launch sites are positioned within 3 km of coastlines, allowing launches over water for safety reasons.

2. **Population Buffer:** Launch sites maintain a safety buffer from population centers, with distances ranging from approximately 15-25 km to the nearest cities.

3. **Transportation Access:** All sites are located within reasonable proximity to major highways, facilitating logistics and equipment transport.

### Launch Success Factors

The success/failure visualization revealed:

1. **Site-Specific Performance:** Some launch sites demonstrate higher success rates than others, suggesting that geographical factors may influence launch outcomes.

2. **Operational Evolution:** The success rates show how SpaceX has improved its launch capabilities over time across all sites.

3. **Strategic Positioning:** The East Coast sites (CCAFS, KSC) leverage Earth's rotation for equatorial orbits, while the West Coast site (VAFB) is positioned for polar orbit insertions.

### Key Insights

1. **Safety and Logistics Balance:** SpaceX has carefully balanced safety considerations (distance from population centers) with logistical practicality (proximity to transportation).

2. **Coastal Requirement:** The consistently short distance to coastlines across all sites highlights the fundamental importance of over-water flight paths.

3. **Success Rate Variation:** The different success rates between sites suggest that geographical factors may play a role beyond just the technology of the rockets themselves.

This analysis demonstrates how geographical factors play a crucial role in the success of a space launch program, extending beyond just the rocket technology itself.

### Next Steps

Building on this geographical analysis, future work could explore:

1. The temporal evolution of launch success rates at each site
2. Correlation between specific mission parameters and site selection
3. Weather patterns at each site and their impact on launch success
4. Comparison with launch sites used by other space agencies and companies

These insights will provide a foundation for the dashboard development in the next phase of this project.